# NLP TO REMOVE GIBBERISH

In [1]:
##Import Libraries

import nltk
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('brown')
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import words, brown
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer, LancasterStemmer
import pandas as pd
from nltk.stem import WordNetLemmatizer
from pathlib import Path

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/philipsurendran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/philipsurendran/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/philipsurendran/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/philipsurendran/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     /Users/philipsurendran/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [2]:
%time

#tokenizer = RegexpTokenizer(r'\w+')

## Import data

df = pd.read_csv('Research/cleaneddata.csv')

## Display head of data

df.head()


# df['words'] = 


lemmatizer=WordNetLemmatizer()
stemmer1 = PorterStemmer()
stemmer2 = LancasterStemmer()

wordlist = set(words.words())
brownlist = set(brown.words())

# filepath = Path('/Users/philipsurendran/Documents/Research/flaggedq19.csv')
# filepath.parent.mkdir(parents=True, exist_ok=True) 
# flagged_2.to_csv(filepath, index=False)



CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.3 µs


In [8]:
def GibberishDetector(variable):
    
    flagged = []
    text = []
    index = df.loc[df[variable].isna() == False]
    
    for i in index[variable]:
        tokens = [word for sent in sent_tokenize(i) for word in word_tokenize(sent)]
        for x in tokens:


            if x.isalpha() == False:
                continue

            elif x.lower() not in wordlist:

                if(lemmatizer.lemmatize(x.lower()) in wordlist):
                    continue
                elif(stemmer1.stem(x.lower()) in wordlist):
                    continue
                elif(stemmer2.stem(x.lower()) in wordlist):
                    continue
                elif(x.lower() == 'african'):
                    continue
                elif(x.lower() in brownlist):
                    continue
                else:
                    rid = index['ResponseId'].loc[index[variable]==i].values[0]
                    flagged.append(rid)
                    text.append(i)
                    break
                    
    flagged_1 = pd.DataFrame({'Response ID': flagged,
                      variable: text}) 

    iterations = flagged_1['Response ID'].size
    iterations = int(iterations/2)
    flagged_2 = []
    
    for i in range(iterations):

        tokens = [word for sent in sent_tokenize(flagged_1[variable][i]) for word in word_tokenize(sent)]

        size = len(tokens)
        real = 0
        for x in tokens:    
            x = x.lower()
            if x.isalpha() == True:
                size-=1
                continue
            if x in wordlist:
                real+=1
                continue
            else:
                if(lemmatizer.lemmatize(x) in wordlist):
                    real+=1
                    continue
                elif(stemmer1.stem(x) in wordlist):
                    real+=1
                    continue
                elif(stemmer2.stem(x) in wordlist):
                    real+=1
                    continue
                elif(x == 'african'):
                    real+=1
                    continue
                elif(x in brownlist):
                    real+=1
        if ((real/(size+1))>0.75):
            continue
        else:
            flagged_2.append([flagged_1['Response ID'][i], flagged_1[variable][i]])

    flagged_final = pd.DataFrame(flagged_2)
    
    flagged_final = flagged_final.rename(columns={0:'Response ID',
                                               1:variable})
                                
    
    return flagged_final

print(GibberishDetector('Q19'))

          Response ID                                                Q19
0   R_2dsXTteXyprbI9g  It wasn’t allowed anymore. People were not all...
1   R_24oTA4bbel9x8SQ                     Unfairly mostly since Jim Crow
2   R_6Jt54MbjEuotrI5               There hasn’t been slavery since then
3   R_1IKp9q8YvPNq1pO  We’ve come a long way but work still needs to ...
4   R_22lB0x3DZ9TOvM6                    it was eradicated after the war
5   R_7OlaqKgJJjV03Pr                 It hasn’t progressed much honestly
6   R_xcshf3oXqZsRlDP  Still exist indirectly thee we truth is not be...
7   R_27D5USH9WTtjZPt       Ended slavery and gave Africans more freedom
8   R_2s7GdwjVjFeXdWh  Has gone into black market such as sex traffic...
9   R_reFAkN6I0IOZCh3                   It hasn’t really been dealt with
10  R_b9Iv1NZk3B53g3v  Slavery is not allowed in this country, The sl...
11  R_5heMLTCiMTTsCFX  It has been outlawed but blacks have been disc...
12  R_3pgcfh0NIMcVF4Q              We’re fighting a

In [18]:
def GibberishAllText(variables, original):
    
    flagged_overall = original
    
    for var in variables:

        new_flagged = GibberishDetector(var)
        flagged_overall = flagged_overall.merge(new_flagged, how='left', on='Response ID')

        temp = flagged_overall.drop(['Response ID'], axis = 1)
        is_NaN = temp.isnull()
        row_has_NaN = is_NaN.all(axis=1)
        flagged_overall = flagged_overall[-row_has_NaN]
        
    
    return flagged_overall

rids = pd.DataFrame({'Response ID': df['ResponseId']})
text_questions = ['Q19','Q20', 'Q39', 'Q41', 'Q42', 'Q53', 'Q54']

final = GibberishAllText(text_questions, rids)

filepath = Path('/Users/philipsurendran/Documents/Research/flaggedtext.csv')
filepath.parent.mkdir(parents=True, exist_ok=True) 
flagged_2.to_csv(filepath, index=False)

          Response ID                                                Q19  \
0   R_2dsXTteXyprbI9g  It wasn’t allowed anymore. People were not all...   
1   R_24oTA4bbel9x8SQ                     Unfairly mostly since Jim Crow   
2   R_6Jt54MbjEuotrI5               There hasn’t been slavery since then   
3   R_1IKp9q8YvPNq1pO  We’ve come a long way but work still needs to ...   
4   R_22lB0x3DZ9TOvM6                    it was eradicated after the war   
5   R_7OlaqKgJJjV03Pr                 It hasn’t progressed much honestly   
6   R_xcshf3oXqZsRlDP  Still exist indirectly thee we truth is not be...   
7   R_27D5USH9WTtjZPt       Ended slavery and gave Africans more freedom   
8   R_2s7GdwjVjFeXdWh  Has gone into black market such as sex traffic...   
9   R_reFAkN6I0IOZCh3                   It hasn’t really been dealt with   
10  R_b9Iv1NZk3B53g3v  Slavery is not allowed in this country, The sl...   
11  R_5heMLTCiMTTsCFX  It has been outlawed but blacks have been disc...   
12  R_3pgcfh